<a href="https://colab.research.google.com/github/aswin24012004/Data-Projects/blob/main/Foodpanda_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ⚙️ 1. Setup PySpark Session

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Foodpanda Analysis").getOrCreate()

In [12]:
# 📥 2. Load the Dataset

df = spark.read.csv('/content/Foodpanda Analysis Dataset.csv',header=True, inferSchema=True)
df.printSchema()
df.show()

root
 |-- customer_id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: string (nullable = true)
 |-- city: string (nullable = true)
 |-- signup_date: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_date: string (nullable = true)
 |-- restaurant_name: string (nullable = true)
 |-- dish_name: string (nullable = true)
 |-- category: string (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- price: double (nullable = true)
 |-- payment_method: string (nullable = true)
 |-- order_frequency: integer (nullable = true)
 |-- last_order_date: string (nullable = true)
 |-- loyalty_points: integer (nullable = true)
 |-- churned: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- rating_date: string (nullable = true)
 |-- delivery_status: string (nullable = true)

+-----------+------+--------+---------+-----------+--------+----------+---------------+---------+-----------+--------+-------+--------------+--------------

In [5]:
df = df.dropDuplicates()
df = df.dropna(subset=["Restaurant_Name", "Rating", "Price", "order_date", "customer_id"])

# Convert numeric columns
df = df.withColumn("Price", col("Price").cast("float")) \
       .withColumn("Rating", col("Rating").cast("float")) \
       .withColumn("quantity", col("quantity").cast("int")) \
       .withColumn("loyalty_points", col("loyalty_points").cast("int")) \
       .withColumn("order_frequency", col("order_frequency").cast("int"))

df.printSchema()
df.show(5)

root
 |-- customer_id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: string (nullable = true)
 |-- city: string (nullable = true)
 |-- signup_date: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_date: string (nullable = true)
 |-- restaurant_name: string (nullable = true)
 |-- dish_name: string (nullable = true)
 |-- category: string (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- Price: float (nullable = true)
 |-- payment_method: string (nullable = true)
 |-- order_frequency: integer (nullable = true)
 |-- last_order_date: string (nullable = true)
 |-- loyalty_points: integer (nullable = true)
 |-- churned: string (nullable = true)
 |-- Rating: float (nullable = true)
 |-- rating_date: string (nullable = true)
 |-- delivery_status: string (nullable = true)

+-----------+------+--------+---------+-----------+--------+----------+---------------+---------+-----------+--------+-------+--------------+---------------+-

In [6]:
# Basic analysis

from pyspark.sql.functions import count, avg, max, min

print("Total coustomer: ", df.count())
print("Unique Customers:", df.select("Customer_id").distinct().count())

Total coustomer:  6000
Unique Customers: 6000


In [7]:


df.select(avg("Price").alias("Avg_Price"), avg("Rating").alias("Avg_Rating")).show()


+-----------------+------------------+
|        Avg_Price|        Avg_Rating|
+-----------------+------------------+
|800.5248802922566|2.9968333333333335|
+-----------------+------------------+



In [8]:
# Top 10 restaurants by total orders
df.groupBy("Restaurant_Name") \
  .agg(count("*").alias("Total_Orders")) \
  .orderBy(col("Total_Orders").desc()) \
  .show(10)

+---------------+------------+
|Restaurant_Name|Total_Orders|
+---------------+------------+
|         Subway|        1260|
|      Pizza Hut|        1224|
|            KFC|        1224|
|    Burger King|        1151|
|     McDonald's|        1141|
+---------------+------------+



In [9]:
# Average spending and rating by city
df.groupBy("city") \
  .agg(avg("Price").alias("Avg_Price"),avg("Rating").alias("Ave_Rating")) \
  .orderBy(col("Avg_Price").desc()) \
  .show(10)

+---------+-----------------+------------------+
|     city|        Avg_Price|        Ave_Rating|
+---------+-----------------+------------------+
| Peshawar|824.3610559950314| 2.993305439330544|
|   Multan| 803.346870756453| 3.004777070063694|
|Islamabad|795.0891744393507|2.9949452401010954|
|   Lahore|793.0045028159926| 2.989317995069844|
|  Karachi|786.1806289806116|  3.00174672489083|
+---------+-----------------+------------------+



In [10]:
# Top rated restaurants
df.groupBy("Restaurant_Name") \
  .agg(avg("Rating").alias("Avg_Rating"), count("*").alias("Num_Reviews")) \
  .orderBy(col("Avg_Rating").desc()) \
  .show(10)

+---------------+------------------+-----------+
|Restaurant_Name|        Avg_Rating|Num_Reviews|
+---------------+------------------+-----------+
|         Subway|3.0873015873015874|       1260|
|     McDonald's|3.0026292725679227|       1141|
|            KFC| 2.974673202614379|       1224|
|    Burger King|2.9661164205039094|       1151|
|      Pizza Hut|2.9493464052287583|       1224|
+---------------+------------------+-----------+



In [11]:
df.describe()

DataFrame[summary: string, customer_id: string, gender: string, age: string, city: string, signup_date: string, order_id: string, order_date: string, restaurant_name: string, dish_name: string, category: string, quantity: string, Price: string, payment_method: string, order_frequency: string, last_order_date: string, loyalty_points: string, churned: string, Rating: string, rating_date: string, delivery_status: string]